In [ ]:
!nvidia-smi

In [ ]:
!pip install ultralytics

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

In [ ]:
import os
HOME = os.getcwd()
print(HOME)

In [ ]:
from ultralytics import YOLO

from IPython.display import display, Image

In [ ]:
os.mkdir(os.path.join(HOME, 'datasets'))
os.mkdir(os.path.join(HOME, 'datasets', 'roboflow'))
%cd {HOME}/datasets/roboflow

!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="FzvHn0RJTWKHAaIDnnSs")
project = rf.workspace("speed-bump-detection").project("speed-bump-detection-se0eh")
dataset = project.version(10).download("yolov8")

In [ ]:
print(dataset.location)

Resizing

In [ ]:
IMAGES_PATH = dataset.location + "/train/images"
RESIZE_IMG_WIDTH = 640
RESIZE_IMG_HEIGHT = 640

for file in os.listdir(IMAGES_PATH):
    img = cv2.imread(file)
    img = cv2.resize(img, (RESIZE_IMG_WIDTH, RESIZE_IMG_HEIGH))
    cv2.imwrite(IMAGES_PATH + "/" + file, img)

## Train Model

In [ ]:
%%writefile {dataset.location}/data.yaml
names:
- Rumble Strip
- Speed-Bump
nc: 2
roboflow:
  license: CC BY 4.0
  project: speed-bump-detection-se0eh
  url: https://universe.roboflow.com/speed-bump-detection/speed-bump-detection-se0eh/dataset/10
  version: 10
  workspace: speed-bump-detection
test: ../test/images
train: ../train/images
val: ../valid/images

In [ ]:
%cd {HOME}

!yolo task=detect mode=train model=yolov8s.pt data={dataset.location}/data.yaml epochs=75 optimizer="SGD" batch=32 lr0=1e-3 imgsz=640 plots=True save=True

In [ ]:
!ls {HOME}/runs/detect/train/

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt

metrics_files = ["confusion_matrix.png", "F1_curve.png", "R_curve.png", "PR_curve.png", "results.png"]
fig, axes = plt.subplots(len(metrics_files), 1, figsize=(40, 30))

axes = axes.flatten()

for ax, metrics_file in zip(axes, metrics_files):
    img = Image.open(f'{HOME}/runs/detect/train/{metrics_file}')
    ax.imshow(img)
    ax.axis('off')  # Hide axes

# Show the plot
plt.show()

In [ ]:
val_batches = ["val_batch0_pred.jpg", "val_batch1_pred.jpg", "val_batch2_pred.jpg"]
fig, axes = plt.subplots(len(val_batches), 1, figsize=(40, 30))

axes = axes.flatten()

for ax, val_batch in zip(axes, val_batches):
    img = Image.open(f'{HOME}/runs/detect/train/{val_batch}')
    ax.imshow(img)
    ax.axis('off')  # Hide axes

# Show the plot
plt.show()

## Validate Model

* **iou** : default(0.6)
* **conf**: default(0.001)

In [ ]:
%cd {HOME}

!yolo task=detect mode=val model={HOME}/runs/detect/train/weights/best.pt data={dataset.location}/data.yaml batch=64 plots=True save_json=True

* **iou** : 0.5
* **conf**: 0.5

In [ ]:
%cd {HOME}

!yolo task=detect mode=val model={HOME}/runs/detect/train/weights/best.pt data={dataset.location}/data.yaml batch=32 conf=0.5 iou=0.5 plots=True save_json=True

* **iou** : 0.7
* **conf**: 0.5

In [ ]:
%cd {HOME}

!yolo task=detect mode=val model={HOME}/runs/detect/train/weights/best.pt data={dataset.location}/data.yaml batch=64 conf=0.5 iou=0.7 plots=True save_json=True

* **iou** : 0.7
* **conf**: 0.6

In [ ]:
%cd {HOME}

!yolo task=detect mode=val model={HOME}/runs/detect/train/weights/best.pt data={dataset.location}/data.yaml batch=64 conf=0.6 iou=0.7 plots=True save_json=True

* **iou** : 0.7
* **conf**: 0.7

In [ ]:
%cd {HOME}

!yolo task=detect mode=val model={HOME}/runs/detect/train/weights/best.pt data={dataset.location}/data.yaml batch=64 conf=0.7 iou=0.7 plots=True save_json=True

## Testing Model

* **iou** : default(0.7)
* **conf**: 0.25

In [ ]:
%cd {HOME}
!yolo task=detect mode=predict model={HOME}/runs/detect/train/weights/best.pt conf=0.25 source={dataset.location}/test/images save=True save_conf=True save_txt=True

In [ ]:
import glob
from IPython.display import Image, display

for image_path in glob.glob(f'{HOME}/runs/detect/predict/*.jpg')[:]:
      display(Image(filename=image_path, width=600))
      print("\n")

* **iou** : 0.5
* **conf**: 0.6

In [ ]:
%cd {HOME}
!yolo task=detect mode=predict model={HOME}/runs/detect/train/weights/best.pt conf=0.6 iou=0.5 source={dataset.location}/test/images save=True save_conf=True save_txt=True

In [ ]:
import glob
from IPython.display import Image, display

for image_path in glob.glob(f'{HOME}/runs/detect/predict2/*.jpg')[:]:
      display(Image(filename=image_path, width=600))
      print("\n")

* **iou** : 0.7
* **conf**: 0.6

In [ ]:
%cd {HOME}
!yolo task=detect mode=predict model={HOME}/runs/detect/train/weights/best.pt conf=0.6 iou=0.7 source={dataset.location}/test/images save=True save_conf=True save_txt=True

In [ ]:
import glob
from IPython.display import Image, display

for image_path in glob.glob(f'{HOME}/runs/detect/predict3/*.jpg')[:]:
      display(Image(filename=image_path, width=600))
      print("\n")

## Export Model

In [ ]:
model = YOLO("yolov8s.pt")
model = YOLO(f"{HOME}/runs/detect/train/weights/best.pt")

model.export(format="onnx")
model.export(format="tflite")